# Import


In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
import math
from sklearn.preprocessing import MinMaxScaler,StandardScaler 
from sklearn.model_selection import train_test_split
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

# Read data

In [25]:
#data = "ALICL_2000-01-01_2021-12-31.csv"
hydro = pd.read_csv("AKJCL_2000-01-01_2021-12-31.csv", engine='python')
hydro.head()


,S.N.,Date,Total Transactions,Total Traded Shares,Total Traded Amount,Max. Price,Min. Price,Close Price
0,1,2021-12-29,293,44404.0,12159732.6,280.0,270.0,273.0
1,2,2021-12-28,474,81741.0,22254586.3,280.0,264.0,274.9
2,3,2021-12-27,660,118407.0,33305303.9,295.4,270.0,272.0
3,4,2021-12-26,608,115254.0,33393501.0,297.0,285.8,289.8
4,5,2021-12-23,819,128123.0,37791022.7,307.0,282.0,291.0


# Sort data by ascending

In [26]:
hydro = hydro.sort_values(by="Date")
hydro.head()

,S.N.,Date,Total Transactions,Total Traded Shares,Total Traded Amount,Max. Price,Min. Price,Close Price
670,671,2018-12-20,75,1245.0,125345.0,138.0,86.0,102.0
669,670,2018-12-23,190,2920.0,295210.0,104.0,98.0,104.0
668,669,2018-12-24,142,2320.0,239350.0,106.0,100.0,102.0
667,668,2018-12-25,35,590.0,58880.0,102.0,98.0,101.0
666,667,2018-12-26,26,390.0,39490.0,102.0,100.0,101.0


# Handling Missing value

In [27]:
hydro = hydro.dropna()
hydro.tail()

,S.N.,Date,Total Transactions,Total Traded Shares,Total Traded Amount,Max. Price,Min. Price,Close Price
4,5,2021-12-23,819,128123.0,37791022.7,307.0,282.0,291.0
3,4,2021-12-26,608,115254.0,33393501.0,297.0,285.8,289.8
2,3,2021-12-27,660,118407.0,33305303.9,295.4,270.0,272.0
1,2,2021-12-28,474,81741.0,22254586.3,280.0,264.0,274.9
0,1,2021-12-29,293,44404.0,12159732.6,280.0,270.0,273.0


# Plot

In [28]:
hydro.iplot(kind="line",x="Date",y="Close Price",xTitle="Date", yTitle="Close Price", title="Hydropower Data Trend")

# Next Price

In [29]:
hydro['Next Price'] = hydro.shift(-1)['Close Price']

In [30]:
hydro = hydro.dropna()

# Rename Columns

In [31]:
hydro.columns

Index(['S.N.', 'Date', 'Total Transactions', 'Total Traded Shares',
       'Total Traded Amount', 'Max. Price', 'Min. Price', 'Close Price',
       'Next Price'],
      dtype='object')

In [32]:
hydro.rename(columns = {'S.N.':'SN','Total Transactions':'TTrans', 'Total Traded Shares':'TTS',
                              'Total Traded Amount':'TTA','Close Price':'ClosePrice','Next Price':'NextPrice',
                     'Max. Price':'MaxPrice','Min. Price':'MinPrice'}, inplace = True)

# Prepared Feature


In [33]:
features = ['ClosePrice',"Date"]
outFeature = ['NextPrice',"Date"]

In [34]:
# take close price as feature and next price as out feature
X, Y = hydro[features], hydro[outFeature] 
X.set_index("Date", inplace=True)
Y.set_index("Date", inplace=True)

# Normalized Data

In [35]:
#Normalize data using standard scalar.
ss = StandardScaler()

X["ClosePrice"] = ss.fit_transform(X)
Y["NextPrice"] = ss.fit_transform(Y)


# Train Test Split

Using train_test_split of sklearn will split data randomly so the sequence is lost.

In [36]:
def train_test_split(x,y,train_size):
    n = int(len(x)*train_size)
    trainx = x[:n]
    testx = x[n:]
    trainy=y[:n]
    testy=y[n:]
    return trainx,testx,trainy,testy

In [37]:
#Split train, validation, test set from X,Y to 80%, 10%, 10%

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.9)

In [38]:
X_train.shape,X_test.shape,X.shape

((603, 1), (67, 1), (670, 1))

# Convert to Matrix


In [39]:
# convert into dataset matrix
def convertToMatrix(data, step):
    data = data.to_numpy()
    X, Y =[], []
    for i in range(len(data)-step):
        d=i+step  
        X.append(data[i:d,])
        Y.append(data[d,])
    return np.array(X), np.array(Y)

step=5

trainX,trainY =convertToMatrix(X_train,step)
testX,testY =convertToMatrix(X_test,step)
# validX,validY = convertToMatrix(X_valid,step)

trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1],1))
# validX = np.reshape(validX, (validX.shape[0], 1, validX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1],1))

trainX.shape,testX.shape #,validX.shape

((598, 5, 1), (62, 5, 1))

# Modeling

In [19]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True,input_shape=(5,1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

In [20]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(trainX, trainY, batch_size=1, epochs=1)

2285/2285 [==============================] - 21s 7ms/step - loss: 0.0207


In [21]:
predictions = model.predict(testX)
predictions = ss.inverse_transform(predictions)
predictions.shape,testX.shape

8/8 [==============================] - 2s 5ms/step


((250, 1), (250, 5, 1))

# RMSE Of Test

In [31]:
Y_test_ss = ss.inverse_transform(Y_test)[:-5]

rmse=np.sqrt(np.mean(((predictions- Y_test_ss)**2)))
print(rmse)

29.22069248967086


# Test Trend

In [32]:
test=pd.DataFrame()
test["pred"]=ss.inverse_transform(predictions.flatten())
test["real"]=ss.inverse_transform(Y_test_ss)
test["date"]=X_test.index[:-5]
test.iplot(kind="line", x="date")